<img width="8%" alt="Pipedrive.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Pipedrive.png" style="border-radius: 15%">

# Pipedrive - Send all persons to a Google Sheets spreadsheet
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Pipedrive+-+Send+all+persons+to+a+Google+Sheets+spreadsheet:+Error+short+description">Bug report</a>

**Tags:** #pipedrive #persons #get #automation #scheduler #gsheet #googlesheets

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-12-28 (Created: 2023-12-28)

**Description:** This notebook send all persons from Pipedrive to a Google Sheets spreadsheet.

**References:**
- [Pipedrive API v1 Documentation](https://developers.pipedrive.com/docs/api/v1/Persons#getPersons)
- [Pipedrive API Authentication](https://developers.pipedrive.com/docs/api/authentication)

## Input

### Import libraries

In [ ]:
import requests
import naas
import pandas as pd
from datetime import date

### Setup variables
**Pre-requisite**

- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com
- [Get your Pipedrive API token](https://developers.pipedrive.com/docs/api/authentication)

**Mandatory**

- `api_token`: Pipedrive API token used to authenticate the request.
- `spreadsheet_url`: Google Sheets spreadsheet URL
- `sheet_name`: Google Sheets sheet name

**Optional**
- `file_path`: CSV file path to save data
- `cron`: CRON syntax to schedule notebooks, find the syntax you need to on: https://crontab.guru/

In [ ]:
# Mandatory
api_token = naas.secret.get("PIPEDRIVE_API_KEY") or "YOUR_API_TOKEN"
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1RdwdYXDFDSFSFxxxxxxxx/edit#gid=XXXXXXXX33"
sheet_name = "MY_SHEET"

# Optional
file_path = f"{date.today().isoformat()}_export_pipedrive_persons.csv"
cron = "0 8 * * *"

## Model

### Get all persons

In [ ]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def get_all_persons(
    api_token,
    start=0,
    limit=100
):
    # Init
    data = []
    
    # Requests
    while True:
        api_url = f'https://api.pipedrive.com/v1/persons?api_token={api_token}&start={start}&limit={limit}'
        res = requests.get(api_url)
        res.raise_for_status()
        if res.status_code == 200:
            res_json = res.json()
        else:
            print(f'Failed to get all organizations. Status code: {res.status_code}')
            break
            
        # Clean data
        if res_json.get("success") and res_json.get('data') is not None:
            for r in res_json.get('data'):
                data.append(flatten_dict(r))
            start += 100
        else:
            break
    return pd.DataFrame(data)
        
df = get_all_persons(api_token)
print("Persons:", len(df))
df.head(5)

## Output

### Save data to CSV

In [ ]:
df.to_csv(file_path, index=False)

### Send data to a Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(
    sheet_name=sheet_name,
    data=df,
    append=False
)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)

# naas.scheduler.delete()